# Semantic Kernel  & Agent-to-Agent (A2A) Protocol

This notebook demonstrates how to implement Agent-to-Agent (A2A) communication using Semantic Kernel. We'll create two agents:

1. **Flight Booking Agent** - A specialized agent that handles flight bookings
2. **Travel Planning Agent** - A general travel agent that uses the flight booking agent as a tool

## What does Agent-to-Agent (A2A) Protocol offer?

A2A Protocol allows different AI agents to work together by calling each other's services. This creates a distributed system where:

- Each agent can specialize in specific tasks
- Agents can leverage other agents' capabilities
- The system becomes more modular and scalable

## Architecture Overview

```
User Request → Travel Planning Agent → Flight Booking Agent → Response
```

The Travel Planning Agent acts as an orchestrator that can handle various travel-related tasks, and when it needs to book flights, it communicates with the specialized Flight Booking Agent through the A2A protocol.


## Prerequisites and Setup

First, let's install the required dependencies and set up our environment.


In [ ]:
# Install required packages
# If you already did the uv configuration, you can skip this step
#%pip install semantic-kernel python-dotenv fastapi uvicorn httpx a2a-sdk

In [ ]:
# Import necessary libraries
import os
import logging
import threading
from uuid import uuid4
from typing import Dict, Any

# Third-party imports
import httpx
import uvicorn
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv('../.env')

print("✅ Dependencies installed and environment configured!")


In [ ]:
# Verify environment variables
required_vars = [
    'AZURE_OPENAI_API_KEY',
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_VERSION',
    'AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'
]

for var in required_vars:
    if os.getenv(var):
        print(f"✅ {var} is set")
    else:
        print(f"❌ {var} is not set")

# Set A2A server URL if not already set
if not os.getenv('A2A_SERVER_URL'):
    os.environ['A2A_SERVER_URL'] = 'http://localhost:9999'
    print("🔧 A2A_SERVER_URL set to default: http://localhost:9999")


## Part 1: Flight Booking Agent Implementation

The Flight Booking Agent is a specialized agent that handles flight booking requests. It understands user requests and maintain conversation context.

### Key Features:
- Specialized in flight booking tasks
- Maintains conversation context per user
- Provides booking confirmation summaries


In [ ]:
# Import Semantic Kernel components
from semantic_kernel.agents.chat_completion.chat_completion_agent import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.chat_history import ChatHistory

class SemanticKernelFlightBookingAgent:
    """A flight booking agent using Semantic Kernel and Azure OpenAI."""

    def __init__(self):
        """Initialize the flight booking agent with Azure OpenAI service."""
        logger.info("Initializing SemanticKernelFlightBookingAgent.")

        self.chat_agent = ChatCompletionAgent(
            service=AzureChatCompletion(),
            name="FlightBookingAssistant",
            instructions=(
                "You are a helpful flight booking assistant. "
                "Your task is to help users book flights by gathering necessary information "
                "such as departure city, destination city, travel dates, number of passengers, "
                "and preferred class of service. Once you have all the required information, "
                "provide a confirmation summary and simulate a successful booking."
            )
        )

        # Store chat history per context to maintain conversation state
        self.history_store: dict[str, ChatHistory] = {}

        logger.info("SemanticKernelFlightBookingAgent initialized successfully.")

    def _get_or_create_chat_history(self, context_id: str) -> ChatHistory:
        """Get existing chat history or create a new one for the given context."""
        chat_history = self.history_store.get(context_id)

        if chat_history is None:
            chat_history = ChatHistory(
                messages=[],
                system_message=(
                    "You are a helpful flight booking assistant. "
                    "Help users book flights by gathering all necessary information: "
                    "departure city, destination city, travel dates, number of passengers, "
                    "and preferred class. Once you have complete information, "
                    "provide a booking confirmation summary."
                )
            )
            self.history_store[context_id] = chat_history
            logger.info(f"Created new ChatHistory for context ID: {context_id}")

        return chat_history

    async def chat(self, user_input: str, context_id: str) -> str:
        """
        Process a flight booking request from the user.

        Args:
            user_input: The user's request for flight booking
            context_id: The context ID for maintaining conversation state

        Returns:
            The response from the flight booking agent
        """
        logger.info(f"Received flight booking request: {user_input} with context ID: {context_id}")

        if not user_input or not user_input.strip():
            logger.error("User input is empty.")
            raise ValueError("User input cannot be empty.")

        try:
            # Get or create chat history for the context
            chat_history = self._get_or_create_chat_history(context_id)

            # Add user input to chat history
            chat_history.messages.append(
                ChatMessageContent(role="user", content=user_input))

            # Create a new thread from the chat history
            thread = ChatHistoryAgentThread(
                chat_history=chat_history, thread_id=str(uuid4()))

            # Get response from the agent
            response = await self.chat_agent.get_response(message=user_input, thread=thread)

            # Add assistant response to chat history
            chat_history.messages.append(ChatMessageContent(
                role="assistant", content=response.content.content))


            logger.info(f"Chat history length is {len(chat_history.messages)} messages for context ID: {context_id}")
            logger.info(f"Flight booking agent response: {response.content.content}")

            return response.content.content

        except Exception as e:
            logger.error(f"Error processing flight booking request: {e}")
            return f"I apologize, but I encountered an error while processing your flight booking request: {str(e)}"

# Create an instance of the flight booking agent
flight_booking_agent = SemanticKernelFlightBookingAgent()
print("✅ Flight Booking Agent created successfully!")


### Test the Flight Booking Agent

Let's test our flight booking agent directly to see how it works:


In [ ]:
response = await flight_booking_agent.chat(  
    user_input="I want to book a flight from New York to London on July 15th", 
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
response = await flight_booking_agent.chat(
    user_input="I need it for 2 passengers, business class", 
    context_id="test_context_1"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
response = await flight_booking_agent.chat(
        "Book a flight to Paris tomorrow", 
        "test_context_2"
    )
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

## Part 2: A2A Server Components

To enable Agent-to-Agent communication, we need to create server components that expose our flight booking agent through the A2A protocol. This involves:

1. **Agent Executor**: Handles A2A protocol integration
2. **Agent Card**: Describes the agent's capabilities
3. **A2A Server**: Exposes the agent via HTTP API

### Agent Executor

The Agent Executor bridges our flight booking agent with the A2A protocol:


In [ ]:
# Import A2A SDK components
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.utils import new_agent_text_message, new_task

class SemanticKernelFlightBookingAgentExecutor(AgentExecutor):
    """Executor for SemanticKernelFlightBookingAgent that handles A2A protocol integration."""

    def __init__(self):
        """Initialize the executor with a flight booking agent instance."""
        logger.info("Initializing SemanticKernelFlightBookingAgentExecutor.")
        self.agent = SemanticKernelFlightBookingAgent()
        logger.info("SemanticKernelFlightBookingAgentExecutor initialized successfully.")

    async def execute(
        self,
        context: RequestContext,
        event_queue: EventQueue,
    ) -> None:
        """
        Execute a flight booking request.

        Args:
            context: The request context containing user input and task information
            event_queue: Queue for sending events and responses
        """
        user_input = context.get_user_input()
        task = context.current_task
        context_id = context.context_id

        # Create a new task if one doesn't exist
        if not task:
            task = new_task(context.message)
            await event_queue.enqueue_event(task)

        logger.info(f"Executing flight booking - User input: {user_input}, Task ID: {task.id}, Context ID: {context_id}")

        try:
            # Process the flight booking request
            result = await self.agent.chat(user_input, context_id)

            # Send the result back through the event queue
            await event_queue.enqueue_event(new_agent_text_message(result))

            logger.info("Flight booking executed successfully.")

        except ValueError as ve:
            logger.error(f"Validation error during flight booking: {ve}")
            await event_queue.enqueue_event(
                new_agent_text_message(f"I need more information to help you book a flight: {str(ve)}")
            )

        except Exception as e:
            logger.error(f"Unexpected error during flight booking execution: {e}")
            await event_queue.enqueue_event(
                new_agent_text_message(
                    "I apologize, but I encountered an error while processing your flight booking request. "
                    "Please try again or contact support if the issue persists."
                )
            )

    async def cancel(
        self,
        context: RequestContext,
        event_queue: EventQueue
    ) -> None:
        """
        Handle cancellation requests.

        Args:
            context: The request context
            event_queue: Queue for sending events
        """
        logger.warning("Cancel operation requested but not supported for flight booking agent.")
        raise Exception('Cancel operation not supported for flight booking operations.')

print("✅ Agent Executor created successfully!")


### A2A Server Configuration

Now we'll create the A2A server that exposes our flight booking agent. This includes:

1. **Agent Card**: Describes the agent's capabilities and skills
2. **Server Application**: Handles HTTP requests and routes them to our agent


In [ ]:
# Import A2A server components
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
)

# Server configuration
SERVER_HOST = "0.0.0.0"
SERVER_PORT = 9999

def create_flight_booking_skill() -> AgentSkill:
    """Create and return the flight booking skill configuration."""
    return AgentSkill(
        id='flight_booking',
        name='Flight Booking',
        description='Assists users in booking flights based on their requests.',
        tags=['flight', 'booking', 'travel'],
        examples=[
            'Book a flight from New York to London next Monday.',
            'I need a flight to Paris tomorrow morning.',
        ],
    )

def create_agent_card() -> AgentCard:
    """Create and return the agent card configuration."""
    return AgentCard(
        name='Semantic Kernel Flight Booking Agent',
        description='An agent that helps users book flights using semantic kernel capabilities.',
        capabilities=AgentCapabilities(streaming=True),
        url=os.environ.get('A2A_SERVER_URL'),
        version='1.0.0',
        defaultInputModes=['text'],
        defaultOutputModes=['text'],
        skills=[create_flight_booking_skill()],
        supportsAuthenticatedExtendedCard=False,
    )

def create_flight_booking_server() -> A2AStarletteApplication:
    """Create and configure the A2A server application."""
    # Initialize request handler with the flight booking agent executor
    request_handler = DefaultRequestHandler(
        agent_executor=SemanticKernelFlightBookingAgentExecutor(),
        task_store=InMemoryTaskStore(),
    )

    # Create and return the server application
    return A2AStarletteApplication(
        agent_card=create_agent_card(),
        http_handler=request_handler,
    )

print("✅ A2A Server configuration created successfully!")


### Start the Flight Booking A2A Server

Now let's start the A2A server that will expose our flight booking agent. This server will run in the background so it doesn't block the rest of our notebook:


In [ ]:
# Function to start the A2A server in a separate thread
def start_a2a_server():
    """Start the A2A server in a separate thread."""
    server = create_flight_booking_server()
    uvicorn.run(server.build(), host=SERVER_HOST, port=SERVER_PORT)

# Start the server in a background thread
server_thread = threading.Thread(target=start_a2a_server, daemon=True)
server_thread.start()


INFO:     127.0.0.1:60818 - "GET /.well-known/agent.json HTTP/1.1" 200 OK


2025-07-11 10:14:12,705 - __main__ - INFO - Executing flight booking - User input: Book a flight from Cairo to Paris departing on November 15th for 5 days, Economy class, for one passenger., Task ID: 73ff7c6f-61f5-40ae-b454-dfca93c1617b, Context ID: travel_booking_context
2025-07-11 10:14:12,706 - __main__ - INFO - Received flight booking request: Book a flight from Cairo to Paris departing on November 15th for 5 days, Economy class, for one passenger. with context ID: travel_booking_context
2025-07-11 10:14:14,514 - httpx - INFO - HTTP Request: POST https://aoai-sweden-gbb-dev.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:14:14,515 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=161, prompt_tokens=2348, total_tokens=2509, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, 

INFO:     127.0.0.1:60818 - "POST / HTTP/1.1" 200 OK


In [ ]:
async with httpx.AsyncClient() as client:
    response = await client.get(f"http://localhost:{SERVER_PORT}/.well-known/agent.json")
    if response.status_code == 200:
        print("✅ A2A Server is running successfully!")
        agent_card = response.json()
        print(f"Agent Name: {agent_card.get('name')}")
        print(f"Agent Description: {agent_card.get('description')}")
        print(f"Agent Skills: {[skill.get('name') for skill in agent_card.get('skills', [])]}")

## Part 3: Travel Planning Agent with A2A Integration

Now we'll create a Travel Planning Agent that can handle various travel-related tasks. When it needs to book flights, it will communicate with our Flight Booking Agent through the A2A protocol.

### Key Features:
- General travel planning capabilities
- Uses the Flight Booking Agent as a tool via A2A
- Demonstrates agent-to-agent communication
- Maintains conversation context


In [ ]:
# Import A2A client components
from a2a.client import A2ACardResolver, A2AClient
from a2a.types import MessageSendParams, SendMessageRequest
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Configuration
FLIGHT_BOOKING_AGENT_URL = os.getenv("A2A_SERVER_URL", "http://localhost:9999")

class FlightBookingTool:
    """Tool for booking flights using the flight booking agent."""

    @kernel_function(
        description="Book a flight using the flight booking agent",
        name="book_flight"
    )
    async def book_flight(self, user_input: str) -> str:
        """
        Book a flight using the external flight booking agent.

        Args:
            user_input: The user's flight booking request

        Returns:
            The response from the flight booking agent
        """
        try:
            async with httpx.AsyncClient() as httpx_client:
                # Resolve the agent card from the flight booking agent
                resolver = A2ACardResolver(
                    httpx_client=httpx_client, 
                    base_url=FLIGHT_BOOKING_AGENT_URL
                )
                agent_card = await resolver.get_agent_card()

                # Create A2A client
                client = A2AClient(
                    httpx_client=httpx_client,
                    agent_card=agent_card
                )

                # Prepare the request
                request = SendMessageRequest(
                    id=str(uuid4()),
                    params=MessageSendParams(
                        message={
                            "messageId": uuid4().hex,
                            "role": "user",
                            "parts": [{"text": user_input}],
                            "contextId": "travel_booking_context",
                        }
                    )
                )

                # Send the message to the flight booking agent
                response = await client.send_message(request)
                result = response.model_dump(mode='json', exclude_none=True)

                logger.info(f"Flight booking tool response: {result}")
                return result["result"]["parts"][0]["text"]

        except Exception as e:
            logger.error(f"Error booking flight: {e}")
            return f"Sorry, I encountered an error while trying to book your flight: {str(e)}"

print("✅ Flight Booking Tool created successfully!")


In [ ]:
# Travel Planning Agent Implementation
def create_travel_agent() -> ChatCompletionAgent:
    """Create and configure the travel planning agent."""
    return ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="TravelPlanner",
        instructions=(
            "You are a helpful travel planning assistant. "
            "Use the provided tools to assist users with their travel plans. "
            "When users ask about flights, use the book_flight tool to help them. "
            "You can also provide general travel advice, recommendations, and assistance "
            "with other travel-related tasks."
        ),
        plugins=[FlightBookingTool()]
    )

# Global chat history store for the travel agent
travel_chat_history_store: dict[str, ChatHistory] = {}

def get_or_create_travel_chat_history(context_id: str) -> ChatHistory:
    """Get existing chat history or create a new one for the given context."""
    chat_history = travel_chat_history_store.get(context_id)

    if chat_history is None:
        chat_history = ChatHistory(
            messages=[],
            system_message=(
                "You are a travel planning assistant. "
                "Your task is to help the user with their travel plans, including booking flights."
            )
        )
        travel_chat_history_store[context_id] = chat_history
        logger.info(f"Created new ChatHistory for context ID: {context_id}")

    return chat_history

# Initialize the travel agent
travel_planning_agent = create_travel_agent()

async def chat_with_travel_agent(user_input: str, context_id: str = "default") -> str:
    """
    Handle chat requests with the travel planning agent.

    Args:
        user_input: The user's message
        context_id: Context identifier for maintaining chat history

    Returns:
        The agent's reply
    """
    logger.info(f"Received travel chat request: {user_input} with context ID: {context_id}")

    try:
        # Get or create chat history for the context
        chat_history = get_or_create_travel_chat_history(context_id)

        # Add user input to chat history
        chat_history.messages.append(
            ChatMessageContent(role="user", content=user_input))

        # Create a new thread from the chat history
        thread = ChatHistoryAgentThread(
            chat_history=chat_history, thread_id=str(uuid4()))

        # Get response from the agent
        response = await travel_planning_agent.get_response(message=user_input, thread=thread)

        # Add assistant response to chat history
        chat_history.messages.append(ChatMessageContent(
            role="assistant", content=response.content.content))

        logger.info(f"Travel agent response: {response.content.content}")

        return response.content.content

    except Exception as e:
        logger.error(f"Error processing travel chat request: {e}")
        return "Sorry, I encountered an error processing your request. Please try again."

print("✅ Travel Planning Agent created successfully!")


### Test the A2A Communication

Now let's test the Agent-to-Agent communication! We'll chat with the Travel Planning Agent, which will use the Flight Booking Agent when needed:


In [ ]:
# Test 1: General travel question (should not trigger flight booking)
print("🧪 Test 1: General travel question")
response = await chat_with_travel_agent(
    "What are some good travel destinations for summer?",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
# Test 2: Flight booking request (should trigger A2A communication)
print("🧪 Test 2: Flight booking request (A2A communication)")
response = await chat_with_travel_agent(
    "I need to book a flight from San Francisco to Tokyo on December 25th",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:

    # Test 3: Follow-up with more flight details (should continue A2A communication)
print("🧪 Test 3: Follow-up with more flight details")
response = await chat_with_travel_agent(
    "Make it for 2 passengers, and I prefer business class",
    "demo_context"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [ ]:
print("🧪 Test 4: Mixed request (travel advice + flight booking)")
response = await chat_with_travel_agent(
    "What's the best time to visit Paris? Also, can you book me a flight there for next month?",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

In [32]:
print("🧪 Test 5: Follow up to finalize the booking)")
response = await chat_with_travel_agent(
    "It's going to be just for myself, departing from Cairo on the 15th of November for 5 days and lets do Economy class",
    "demo_context2"
)
print("-" * 50)
print("Final Response")
print("-" * 50)
print(f"Agent: {response}")
print("-" * 50)

2025-07-11 10:14:11,808 - __main__ - INFO - Received travel chat request: It's going to be just for myself, departing from Cairo on the 15th of November for 5 days and lets do Economy class with context ID: demo_context2


🧪 Test 5: Follow up to finalize the booking)


2025-07-11 10:14:12,659 - httpx - INFO - HTTP Request: POST https://aoai-sweden-gbb-dev.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
2025-07-11 10:14:12,661 - semantic_kernel.connectors.ai.open_ai.services.open_ai_handler - INFO - OpenAI usage: CompletionUsage(completion_tokens=45, prompt_tokens=1387, total_tokens=1432, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-07-11 10:14:12,662 - semantic_kernel.connectors.ai.chat_completion_client_base - INFO - processing 1 tool calls in parallel.
2025-07-11 10:14:12,662 - semantic_kernel.kernel - INFO - Calling FlightBookingTool-book_flight function with args: {"user_input":"Book a flight from Cairo to Paris departing on November 15th for 5 days, Economy class, for one passenger."}
2025-07-11 10:14:12,6

--------------------------------------------------
Final Response
--------------------------------------------------
Agent: Your flight from Cairo to Paris, departing on November 15th and returning on November 20th, for one passenger in Economy Class has been successfully confirmed!

You will receive additional details, including your itinerary and e-tickets, via email shortly. If there are any more questions or further assistance is needed, feel free to ask. Safe travels!
--------------------------------------------------


### Interactive Chat

You can now interact with the Travel Planning Agent directly. Try asking about travel destinations, flight bookings, or any travel-related questions:


In [33]:
# Interactive Chat Function
async def interactive_chat():
    """Interactive chat with the Travel Planning Agent."""
    print("💬 Starting interactive chat with Travel Planning Agent")
    print("Type 'exit' to end the conversation")
    print("-" * 50)
    
    context_id = "interactive_session"
    
    while True:
        try:
            # In a real Jupyter environment, you might want to use input() or ipywidgets
            user_input = input("You: ")
            
            if user_input.lower() in ['exit', 'quit', 'bye']:
                print("👋 Thank you for chatting! Goodbye!")
                break
                
            if user_input.strip():
                response = await chat_with_travel_agent(user_input, context_id)
                print(f"Travel Agent: {response}")
                print("-" * 50)
                
        except KeyboardInterrupt:
            print("\n👋 Chat ended by user. Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")
            print("-" * 50)

# Example usage (uncomment to run interactively)
# await interactive_chat()

# For demonstration, let's show some example interactions
print("💡 Example interactions you can try:")
print("- 'What are the best beaches in Thailand?'")
print("- 'Book a flight from London to New York tomorrow'")
print("- 'What's the weather like in Tokyo in spring?'")
print("- 'I need a round-trip flight to Paris for 2 people'")
print("- 'Tell me about travel insurance options'")
print("")
print("💬 To start interactive chat, uncomment and run: await interactive_chat()")


💡 Example interactions you can try:
- 'What are the best beaches in Thailand?'
- 'Book a flight from London to New York tomorrow'
- 'What's the weather like in Tokyo in spring?'
- 'I need a round-trip flight to Paris for 2 people'
- 'Tell me about travel insurance options'

💬 To start interactive chat, uncomment and run: await interactive_chat()


## Summary and Key Concepts

Congratulations! You've successfully implemented Agent-to-Agent (A2A) communication using Semantic Kernel. Here's what we accomplished:

### 🎯 What We Built

1. **Flight Booking Agent**: A specialized agent that handles flight bookings
2. **A2A Server**: Exposes the flight booking agent via HTTP API
3. **Travel Planning Agent**: A general travel agent that uses the flight booking agent as a tool
4. **A2A Communication**: Seamless communication between agents

### 🔑 Key Concepts Learned

#### 1. **Agent Specialization**
- Each agent has a specific purpose and set of capabilities
- Specialization leads to better performance and maintainability
- Agents can be developed and deployed independently

#### 2. **A2A Protocol**
- Standardized way for agents to communicate
- Includes agent cards, skill definitions, and message formats
- Enables discovery and invocation of agent capabilities

#### 3. **Tool Integration**
- Agents can use other agents as tools
- Semantic Kernel's function calling mechanism works with A2A
- Allows for complex workflows and orchestration

#### 4. **Context Management**
- Each agent maintains its own conversation context
- Context IDs ensure proper conversation tracking
- Enables stateful interactions across agent boundaries

### 🏗️ Architecture Benefits

1. **Modularity**: Each agent can be developed, tested, and deployed separately
2. **Scalability**: Agents can be scaled independently based on demand
3. **Reusability**: Specialized agents can be used by multiple orchestrators
4. **Maintainability**: Clear separation of concerns makes code easier to maintain

### 🚀 Next Steps

To extend this implementation, you could:

1. **Add More Specialized Agents**: Hotel booking, car rental, restaurant recommendations
2. **Implement Authentication**: Add security to agent communications
3. **Add Monitoring**: Track agent performance and usage metrics
4. **Deploy to Production**: Use container orchestration for scalability
5. **Add More Tools**: Integrate with real booking APIs and services

### 💡 Best Practices

1. **Design for Single Responsibility**: Each agent should have a clear, focused purpose
2. **Use Descriptive Agent Cards**: Help other agents understand capabilities
3. **Implement Error Handling**: Handle network failures and agent unavailability
4. **Monitor Agent Health**: Ensure agents are responsive and functioning correctly
5. **Version Your Agents**: Use semantic versioning for agent APIs

This tutorial demonstrates the power of agent-to-agent communication in creating sophisticated, distributed AI systems. The modular approach allows for building complex workflows while maintaining clean, maintainable code."


## Optional: Cleanup

If you want to stop the A2A server that's running in the background, you can run the following code:


In [ ]:
# Optional: Cleanup
# Note: The server thread is running as a daemon thread, so it will automatically
# terminate when the main Python process ends. If you want to explicitly check
# if the server is still running, you can use:

if server_thread.is_alive():
    print("✅ A2A Server is still running in the background")
    print("🔧 The server will automatically stop when the notebook kernel is restarted")
else:
    print("❌ A2A Server thread has stopped")

# You can also check the server status again
print("\n🔍 Final server status check:")
await check_server_status()

print("\n🎉 Tutorial completed successfully!")
print("💡 Remember: The A2A server runs as a daemon thread and will stop automatically when the notebook kernel is restarted.")


## Running the Standalone Agent Files

This tutorial includes standalone implementations of both agents that can be run independently:

### Flight Booking Agent Server
The flight booking agent is available as a standalone server in the `flight-booking-agent/` directory:

```bash
cd flight-booking-agent/
uv run server.py
```

This will start the A2A server on `http://localhost:9999` with the flight booking agent.

### Travel Booking Agent
The travel booking agent with a web interface is available in the `travel-booking-agent/` directory:

```bash
cd travel-booking-agent/
uv run agent.py
```

This will start a local web server where you can interact with the travel planning agent through a browser interface.